Visualizer for simulated dynamic adaptive networks

Access: datafiles with adj matrices+culture vecs

In [1]:
%matplotlib widget
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import analysis as analysis
from analysis import read_pickle
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [2]:
filename = 'base' #'mod'
#df = analysis.read_pickle('df_sayama_change_all')
#df = read_pickle('df_group_size10-40_changeall')
df_full = analysis.read_pickle('df_' + filename)
print(df_full.shape)
df_full.columns

(21600, 18)


Index(['degrees', 'clusterCoeff', 'reciprocity', 'center_1', 'center_2',
       'mean_centers', 'overall_mean_culture', 'giantComponent', 'diam', 'SPL',
       'CD', 'c1_init', 'c2_init', 'c_avg_init', 'std_d', 'std_rs', 'std_rw',
       'tags'],
      dtype='object')

## Number of experiments vs Size of Largest Component

In [ ]:
x = []
y = []
y2 = []
for i in np.linspace(0, 1.0, 21):
    cum = 1 - i
    con = df_full['sc'] >= cum
    con2 = df_full['giantComponent'] >= cum
    x.append(cum)
    y.append(df_full.loc[con].shape[0])
    y2.append(df_full.loc[con].shape[0])
for i in np.linspace(0,0.15,15):
    cum = 1 - i
    con = df_full['sc'] >= cum
    con2 = df_full['giantComponent'] >= cum
    x.append(cum)
    y.append(df_full.loc[con].shape[0])
    y2.append(df_full.loc[con].shape[0])

plt.close('all')
plt.scatter(x,y)
plt.show()
plt.scatter(x,y2)
plt.show()

Start with 0.8 as minimum gc size. Filter df:

In [3]:
df = df_full.loc[df_full['sc'] >= 0.0]
print(df_full.shape, df.shape)


KeyError: 'sc'

## Plot base sayama graphs and do linreg

In [ ]:
#mplot3d doesnt handle sympy floats
stdd = np.asarray(df['std_d'].values, dtype = "float")
stdrs = np.asarray(df['std_rs'].values, dtype = "float")
stdrw = np.asarray(df['std_rw'].values, dtype = "float")
spl = np.asarray(df['SPL'].values, dtype = "float")
cd = np.asarray(df['CD'].values, dtype = "float")
cc = np.asarray(df['giantComponent'].values, dtype = "float")
sc = np.asarray(df['sc'].values, dtype = "float")

analysis.plotfig(stdd, cd, spl, 'd', 'base_d')
analysis.plotfig(stdrw, cd, spl, '$r_w$', 'base_rw')
analysis.plotfig(stdrs, cd, spl, '$r_s$', 'base_rs')

In [ ]:
df_full['SPL'].min()

In [ ]:
def asnp(series):
    return np.asarray(series.values, dtype = "float")
def ddgroupby(df, labels):
    sdf = pd.DataFrame(df.groupby(labels)['sc'].mean())
    sdf.reset_index(inplace=True)
    return sdf
def plt_surf(label, df, sdf):
    plt.close('all')
    fig = plt.figure()
    fig.set_size_inches(9, 9)
    ax = fig.gca(projection='3d')

    ax.scatter(asnp(df[label1[0]]), asnp(df[label1[1]]), asnp(df['sc']), marker='.')
    ax.plot_trisurf(asnp(sdf[label[0]]), asnp(sdf[label[1]]), asnp(sdf['sc']),
                    cmap=cm.coolwarm, linewidth=0, edgecolor='none')
    ax.set_xlabel(label[0])
    plt.draw()

In [ ]:
label1 = ['std_rw', 'std_rs']
label2 = ['std_d', 'std_rw']
label3 = ['std_d', 'std_rs']
labels = label2

sdf = ddgroupby(df, labels)
plt_surf(labels, df, sdf)

In [ ]:
sdf.head()

In [ ]:
#todo plot in right order for overlap
analysis.plotfig(stdd, stdrw, sc, '$d$',fsize=(10,10))
#analysis.plotfig(stdrs, cd, spl, '$r_s$', 'base_rs')
#analysis.plotfig(stdrw, cd, spl, '$r_w$', 'base_rw')

## discrepancy in SPL

In [ ]:
#load SPL on undirected conversion of graph
filename = 'base' + 'mod'
dfmod = analysis.read_pickle('df_' + filename)
dfmod.columns

In [ ]:
fig = plt.figure()
fig.set_size_inches(11,11)
ax = fig.add_subplot(111, projection='3d')

ax.scatter(stdd, stdrw, spl, c='b', marker='.')
ax.scatter(asnp(dfmod['std_d']), asnp(dfmod['CD']), asnp(dfmod['SPL']),
          c='r', marker='x')
ax.set_xlabel("s.d. of $d$")
ax.set_ylabel("<CD>")
ax.set_zlabel("<SPL>")
ax.view_init(elev=305, azim=27)
plt.draw()